In [1]:
# This code block is for automatic testing purposes, please ignore.
try:
    import openfermion
except:
    import os
    os.chdir('../src/')

# Demonstration of Givens rotation decomposition

In [2]:
import numpy
from openfermion.utils import givens_decomposition
from scipy.linalg import qr

numpy.set_printoptions(precision=3, linewidth=150, suppress=True)

A Slater determinant can be represented by an $m \times n$ matrix with orthonormal rows. Let's obtain such a matrix $Q$ using Scipy's QR decomposition algorithm.

In [3]:
m, n = (3, 6)

# Obtain a random matrix of orthonormal rows
x = numpy.random.randn(n, n)
y = numpy.random.randn(n, n)
A = x + 1j*y
Q, R = qr(A)
Q = Q[:m, :]
print(Q)

[[-0.134+0.446j  0.018+0.271j  0.703+0.294j  0.025-0.198j -0.074+0.061j  0.267-0.091j]
 [ 0.336+0.267j -0.441-0.245j -0.077-0.02j   0.343+0.422j  0.083+0.215j  0.434-0.131j]
 [-0.395-0.103j  0.001-0.253j -0.215-0.216j -0.058-0.47j  -0.138+0.258j  0.462-0.392j]]


There exist unitary matrices $V$ and $U$ such that $VQU^\dagger$ is an $m \times n$ diagonal matrix. The matrix $VQ$ represents the same Slater determinant as $Q$ up to an overall phase (equal to $\det(V)$), but it has zeros in the upper diagonal, which saves some work in the preparation of the Slater determinant. The unitary $U$ can be written in the form

$$U = G_{N_G} \cdots G_2 G_1$$

where the $G_k$ are complex Givens rotations. We can obtain $V$, the $G_k$, and the diagonal entries of $VQU^\dagger$ using the `givens_decomposition` function.

In [4]:
# Get Givens decomposition of U
givens_rotations, V, diagonal = givens_decomposition(Q)
print(V)
print()
print(diagonal)

[[ 0.912+0.068j -0.077+0.112j -0.297-0.239j]
 [ 0.181-0.096j -0.806-0.128j  0.420+0.339j]
 [ 0.349+0.j     0.561-0.019j  0.698+0.276j]]
()
[-0.209+0.978j -0.714-0.7j   -0.645+0.764j]


We didn't print the Givens rotations in the above block because it wouldn't look pretty. The Givens rotations are returned as a list of tuples of tuples. We will now iterate through the tuples of the list, and print the innermost tuple within each tuple as a string.

In [5]:
for parallel_set in givens_rotations:
    print(tuple(["{}, {}, {:.3f}, {:.3f}".format(i, j, theta, phi)
                 for i, j, theta, phi in parallel_set]))

('2, 3, 0.172, -0.438',)
('1, 2, 1.212, 2.149', '3, 4, 0.313, 1.750')
('0, 1, 0.986, 2.937', '2, 3, 1.062, 1.518', '4, 5, 1.106, 2.647')
('1, 2, 0.945, -0.406', '3, 4, 1.190, -0.769')
('2, 3, 1.339, -1.335',)


There are 5 tuples printed, and within each tuple, there are strings of the form 'i, j, theta, phi'. Each such string represents an innermost tuple, and it is a description of a complex Givens rotation of the coordinates $i$ and $j$ by angles $\theta$ and $\varphi$. The $2 \times 2$ matrix corresponding to this rotation is

$$
\begin{pmatrix}
\cos \theta & -e^{i \varphi} \sin \theta \\
\sin \theta & e^{i \varphi} \cos \theta
\end{pmatrix}.
$$

The fact that there are 5 tuples means that the circuit depth to prepare the Slater determinant corresponding to $Q$ (up to a phase) has depth 5. All of the rotations within the tuple can be performed in parallel; this is possible because the indices to be rotated are disjoint. For instance, in the third step, we can perform three rotations simultaneously, on coordinates $(0, 1)$, $(2, 3)$, and $(4, 5)$.

Let's check that $VQ$ has zeros in the upper right corner.

In [6]:
# Check that VQ has zeros in upper right corner
W = V.dot(Q)
print(W)

[[-0.115+0.54j  -0.002+0.292j  0.642+0.425j -0.132-0.02j  -0.000+0.j     0.000-0.j   ]
 [-0.349-0.342j  0.440+0.195j  0.198-0.152j -0.102-0.64j  -0.192-0.104j  0.000+0.j   ]
 [-0.100+0.118j -0.176-0.211j  0.111-0.117j  0.298-0.182j -0.142+0.282j  0.765-0.26j ]]


Now let's check the Givens decomposition. For each set of Givens rotations that can be performed in parallel, we construct the matrices corresponding to the Givens rotations and multiply them together. Then, we multiply $W = VQ$ repeatedly on the right by these matrices and check that the correct elements are zeroed out.

In [7]:
# Check the Givens decomposition
def expanded_givens(G, i, j, n):
    expanded_G = numpy.eye(n, dtype=complex)
    expanded_G[([i], [j]), (i, j)] = G
    return expanded_G

U = numpy.eye(n, dtype=complex)
for parallel_set in givens_rotations:
    print("Number of rotations to perform in parallel: {}".format(len(parallel_set)))
    combined_givens = numpy.eye(n, dtype=complex)
    for i, j, theta, phi in parallel_set:
        c = numpy.cos(theta)
        s = numpy.sin(theta)
        phase = numpy.exp(1.j * phi)
        G = numpy.array([[c, -phase * s],
                     [s, phase * c]], dtype=complex)
        expanded_G = expanded_givens(G, i, j, n)
        combined_givens = combined_givens.dot(expanded_G)
    W = W.dot(combined_givens.T.conj())
    U = combined_givens.dot(U)
    print(W)
    print()

Number of rotations to perform in parallel: 1
[[-0.115+0.54j  -0.002+0.292j  0.651+0.431j  0.000+0.j    -0.000+0.j     0.000-0.j   ]
 [-0.349-0.342j  0.440+0.195j  0.164-0.043j  0.210-0.64j  -0.192-0.104j  0.000+0.j   ]
 [-0.100+0.118j -0.176-0.211j  0.050-0.108j  0.361-0.058j -0.142+0.282j  0.765-0.26j ]]
()
Number of rotations to perform in parallel: 2
[[-0.115+0.54j  -0.005+0.834j -0.000+0.j     0.000-0.j     0.000+0.j     0.000-0.j   ]
 [-0.349-0.342j  0.272+0.175j  0.368+0.143j  0.221-0.672j  0.000+0.j     0.000+0.j   ]
 [-0.100+0.118j  0.049-0.09j  -0.206-0.192j  0.250-0.083j  0.400+0.068j  0.765-0.26j ]]
()
Number of rotations to perform in parallel: 3
[[-0.209+0.978j -0.000+0.j    -0.000+0.j    -0.000-0.j     0.000+0.j     0.000-0.j   ]
 [ 0.000-0.j    -0.418-0.41j   0.756+0.293j  0.000-0.j     0.000+0.j     0.000+0.j   ]
 [ 0.000-0.j    -0.120+0.142j -0.039+0.129j -0.214-0.291j  0.891+0.151j -0.000+0.j   ]]
()
Number of rotations to perform in parallel: 2
[[-0.209+0.978j  0.00

Finally, let's check that the final matrix, $VQU^\dagger$, is indeed diagonal, and that its entries match the ones returned by the function.

In [8]:
# Check the diagonal entries
D = numpy.zeros((m, n), dtype=complex)
D[numpy.diag_indices(m)] = diagonal
print("V * Q * U^\dagger matches the returned diagonal:")
print(numpy.all(numpy.isclose(D, V.dot(Q.dot(U.T.conj())))))

V * Q * U^\dagger matches the returned diagonal:
True
